In [2]:
# projectpath = "./"
projectpath = "../"
librarypath = "../BackAction.jl"
import Pkg
Pkg.activate(projectpath)
Pkg.develop(path=librarypath)
Pkg.add(["DifferentialEquations", "LinearAlgebra",
        "Distributed", "DelimitedFiles", "TOML"])
Pkg.instantiate()
Pkg.precompile()
Pkg.resolve()
using DifferentialEquations, LinearAlgebra, Distributed,  DelimitedFiles, TOML
import BackAction.Utilities as util
import BackAction.MonteCarloWaveFunction as mc
import BackAction.CoreStructs as cs



  Activating project at `~/Documents/Research/MonitoringMetrology/notebooks`
   Resolving package versions...
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/notebooks/Manifest.toml`


In [2]:
using Statistics

In [3]:
include("Code/constants.jl")

1

In [4]:
include("Code/functions.jl")

extractexpvalsample! (generic function with 1 method)

In [5]:
L0 = d -> sqrt(GAMMA) * util.sigma_m
H0 = d -> d * [[0, 0] [0, 1.0 + 0im]] + 0.5 * OMEGA * util.sigma_x

#5 (generic function with 1 method)

In [6]:
alpha = [0.0+0.0im]

1-element Vector{ComplexF64}:
 0.0 + 0.0im

In [7]:
function obtain_ensol(L0, H0, delta, T, alpha, params, tspan, e_ops, tlist)
        return mc.get_sol_jumps(cs.System(H0(delta), L0(delta), T, alpha),
                                params, tspan, e_ops, tlist;
                                save_on=false, save_start=false, save_end=false)
end

obtain_ensol (generic function with 2 methods)

In [8]:
sim = obtain_ensol(L0, H0, cfg["DELTA"], T, alpha, params, tspan, e_ops, tlist);

In [18]:
# Extract currents
ntimes = cfg["ntimes"] - 1
currentsample = zeros(Float64, ntimes, cfg["ntraj"]) 
# the idea is to count how many jumps there are in each time bin
@time for trajectoryindex in 1:cfg["ntraj"]
    jumptimes = sim[trajectoryindex].prob.kwargs[:callback].continuous_callbacks[1].affect!.jump_times
    njumps = length(jumptimes)
    if njumps == 0
        continue
    end 
    jumpindex = 1
    timebinindex = 1
    #trajectoryindex = 1
    while (timebinindex <= ntimes) & (jumpindex <= njumps )
        if tlist[timebinindex] <= jumptimes[jumpindex] <= tlist[timebinindex+1]
                    currentsample[timebinindex, trajectoryindex] += 1.0 
                    jumpindex += 1
                    continue
        end 
        timebinindex += 1
    end

end
# Write the current sample 
writedlm("currentsample_$(real(alpha[1]))+$(imag(alpha[1])).csv", currentsample, ',')

  0.020331 seconds (452.06 k allocations: 9.187 MiB)


In [12]:
# Now begin to get the statistics!
currentmean = mean(currentsample, dims=2);

In [13]:
dt = tf/cfg["ntimes"];

In [14]:
rsample = Array{ComplexF64}(undef, nops, cfg["ntimes"], cfg["ntraj"])
extractexpvalsample!(rsample, sim)


In [15]:
rmean = dropdims(mean(rsample, dims=3), dims=3)

3×200 Matrix{ComplexF64}:
 0.0+0.0im        0.0+0.0im        0.0+0.0im  …          0.0+0.0im
 0.0+0.0im  -0.312668+0.0im  -0.567825+0.0im       0.0915976+0.0im
 1.0+0.0im   0.880961+0.0im    0.67868+0.0im     0.000299262+0.0im

In [16]:
using Plots
plot(tlist[1:ntimes], currentmean/dt , label="Average Current", title="Direct Photodetection")
plot!(tlist, real(0.5*cfg["GAMMA"]*(rmean[3, :] .+ 1)), label="Analytical Value")
savefig("CurrentDirectPhotoDetection.pdf")
#plot!([tlist[1], tlist[end]], [0.0, 0.0], color="red", label="Diffusive limit, Analytical ")

"/home/ste1nb0ck/Documents/Research/MonitoringMetrology/notebooks/JumpTimeSample/CurrentDirectPhotoDetection.pdf"

In [17]:
# Autocorrelation Function

In [3]:
using Mmap, TOML

In [5]:
ntimes

199

In [6]:
ntraj

1000

In [9]:
cfg = TOML.parsefile("config.toml")
ntimes = cfg["ntimes"] - 1
ntraj = 10_000
filename = "currentsample.bin"
currentsample = open(filename, "r") do io
    Mmap.mmap(io, Matrix{Float64}, (ntimes, ntraj))
end

199×10000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  …  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [11]:
include("Code/constants.jl")

4

In [13]:
using Plots, Statistics

In [14]:
dt = tf/cfg["ntimes"];
currentmean = mean(currentsample, dims=2);

In [15]:
plot(tlist[1:ntimes], currentmean/dt , label="Average Current", title="Direct Photodetection")
#plot!(tlist, real(0.5*cfg["GAMMA"]*(rmean[3, :] .+ 1)), label="Analytical Value")
savefig("CurrentDirectPhotoDetection.pdf")

"/home/ste1nb0ck/Documents/Research/MonitoringMetrology/notebooks/JumpTimeSample/CurrentDirectPhotoDetection.pdf"